# File Generator

This notebook generates all necessary files for the modulytics site. Capitalized comments indicate things that may need to be changed, and lowercase comments just describe what the code is doing.

For more instructions, see [the wiki](https://github.com/SBRG/modulytics/wiki/Adding-a-Private-Project).

Be very sure to start with this file in the right place! Instructions are in the wiki.

## Load ICA data

If you followed the instructions in the wiki, the only path you should need to modify below is the path to ICA_CODE_DIR. It needs to point to a folder containing [the ICA codebase](https://github.com/SBRG/ICA).

The cutoff is determined in the notebook [3_estimate_thresholds.ipynb](https://github.com/SBRG/ICA/blob/master/notebooks/3_estimate_thresholds.ipynb). The organism name only matters if it is 'ecoli', but fill it in regardless. 

The base_conds are the conditions that you used as a baseline for normalization. They are only used here to get the x-axis values in the gene scatter plot.

In [54]:
# first, make sure the template is in the right place.
# ENSURE THIS OUTPUT IS CORRECT
import os
cwd = os.getcwd().split('/')
assert(cwd[-3]=='organisms')
print('Organism Folder: ', cwd[-2])
print('Dataset Folder: ', cwd[-1])

Organism Folder:  e_coli
Dataset Folder:  ros


In [16]:
# ENSURE THIS PATH IS CORRECT
ICA_CODE_DIR = '../../../../../main/ica/'

# ADD THIS INFORMATION
cutoff = 600 
organism = 'ecoli'
base_conds = ['control__wt_glc__1','control__wt_glc__2']

# other paths
DATA_DIR = 'data_files/'
GENE_DIR = '../annotation/gene_files/'
PY_DIR = '../../../py/'

In [2]:
# load ICA codebase
import sys, warnings
sys.path.append(ICA_CODE_DIR)
try:
    from icaviz.plotting import *
except:
    warnings.warn('Path to ICA codebase is incorrect.')

In [3]:
# load data
ica_data = load_data(X=DATA_DIR+'log_tpm.csv', # NOT normalized
                     S=DATA_DIR+'M.csv', # may be called 'S.csv'
                     A=DATA_DIR+'A.csv',
                     metadata=DATA_DIR+'sample_metadata.csv',
                     cutoff=cutoff, organism=organism,
                     
                     # if you do not have the below info, contact Kevin.
                     annotation=GENE_DIR+'gene_info.csv',
                     trn=GENE_DIR+'TRN.csv')

# load the curated_enrichments file
enrich = pd.read_csv(DATA_DIR+'curated_enrichments.csv', index_col=0)

In [4]:
# load py files
sys.path.append(PY_DIR)
from gene_table import *
from gene_histogram import *
from gene_scatter import *
from activity_bar import *
from gene_activity_bar import *
from regulon_venn import *
from regulon_scatter import *
from general_functions import *

## Check that the files are formatted correctly

### Metadata file (DATA_DIR+'sample_metadata.csv')

This lists each sample (indexed the same as X and A) and must contain a condition_id and project_id column. The project_ids define groups that will be separated by vertical lines in the A matrix plot -- if you are working on the modulome, these should separate each BioProject and be named uniquely and descriptively (e.g. 'phage_infection', 'antibiotic_panel', etc). The condition_ids should describe the condition (e.g. 'wt_ctrl', 'kanamycin_supp'). Each pair of project_ids and condition_ids should define a single biological condition. Therefore, you can have 'wt_ctrl' in multiple projects, but any samples that share the same condition_id and project_id must be biological replicates from the same condition. Any additional columns will be accessible within the site, but are not required.

In [5]:
# ADDRESS THESE IF THEY FAIL
# can find correct columns
assert('condition_id' in ica_data.metadata.columns)
assert('project_id' in ica_data.metadata.columns)

# can match to the data
assert(ica_data.X.columns.isin(ica_data.metadata.index).all())
assert(ica_data.A.columns.isin(ica_data.metadata.index).all())

To ensure that the labels are correct, run the following code block. You will see separate tables of metadata for each set of biological replicates. Make sure that all samples listed in each table are biological replicates of each other. If not, edit the metadata file with new condition_ids.

In [6]:
# CHECK THAT THIS GROUPING IS CORRECT
for name, group in ica_data.metadata.groupby(['project_id', 'condition_id']):
    display(group)

,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
X42c__42c_ale1__1,42C Evolution,42c,42c_ale1,1,Ryan LaCroix,Richard Szubin,5/7/2013 0:00,Escherichia coli 42C.1.124.1,MG1655,M9,...,MiSeq,42C A1 F124 I1,1,42c/ALE-42-glu-end-point-Clone-1_S1_L001_R1_00...,42c/ALE-42-glu-end-point-Clone-1_S1_L001_R2_00...,42c__42c_ale1__1.bam,98.4,doi.org/10.1093/molbev/msu209,GSE132442,1.0


,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
X42c__42c_ale10__1,42C Evolution,42c,42c_ale10,1,Ryan LaCroix,Richard Szubin,5/3/2013 0:00,Escherichia coli 42C.10.153.1,MG1655,M9,...,MiSeq,42C A10 F153 I1,1,42c/ALE-42-glu-end-point-Clone-10_S2_L001_R1_0...,42c/ALE-42-glu-end-point-Clone-10_S2_L001_R2_0...,42c__42c_ale10__1.bam,96.91,doi.org/10.1093/molbev/msu209,GSE132442,1.0


,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
X42c__42c_ale2__1,42C Evolution,42c,42c_ale2,1,Ryan LaCroix,Richard Szubin,5/2/2013 0:00,Escherichia coli 42C.2.163.1,MG1655,M9,...,MiSeq,42C A2 F163 I1,1,42c/ALE-42-glu-end-point-Clone-2_S1_L001_R1_00...,42c/ALE-42-glu-end-point-Clone-2_S1_L001_R2_00...,42c__42c_ale2__1.bam,97.51,doi.org/10.1093/molbev/msu209,GSE132442,1.0


,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
X42c__42c_ale3__1,42C Evolution,42c,42c_ale3,1,Ryan LaCroix,Richard Szubin,5/1/2013 0:00,Escherichia coli 42C.3.120.1,MG1655,M9,...,MiSeq,42C A3 F120 I1,1,42c/ALE-42-glu-end-point-Clone-3_S2_L001_R1_00...,42c/ALE-42-glu-end-point-Clone-3_S2_L001_R2_00...,42c__42c_ale3__1.bam,97.13,doi.org/10.1093/molbev/msu209,GSE132442,1.0


,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
X42c__42c_ale4__1,42C Evolution,42c,42c_ale4,1,Ryan LaCroix,Richard Szubin,5/2/2013 0:00,Escherichia coli 42C.4.161.1,MG1655,M9,...,MiSeq,42C A4 F161 I1,1,42c/ALE-42-glu-end-point-Clone-4_S1_L001_R1_00...,42c/ALE-42-glu-end-point-Clone-4_S1_L001_R2_00...,42c__42c_ale4__1.bam,97.07,doi.org/10.1093/molbev/msu209,GSE132442,1.0


...

,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
ytf__delyiaJ__1,yTF,ytf,delyiaJ,1,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655 del_yiaJ,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR6782003_1.fastq.gz,ytf/SRR6782003_2.fastq.gz,ytf__delyiaJ__1.bam,94.63,doi.org/10.1093/nar/gky752,GSE111094,1.0
ytf__delyiaJ__2,yTF,ytf,delyiaJ,2,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655 del_yiaJ,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR6782004_1.fastq.gz,ytf/SRR6782004_2.fastq.gz,ytf__delyiaJ__2.bam,95.07,doi.org/10.1093/nar/gky752,GSE111094,1.0


,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
ytf__delyieP__1,yTF,ytf,delyieP,1,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655 del_yieP,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR6782005_1.fastq.gz,ytf/SRR6782005_2.fastq.gz,ytf__delyieP__1.bam,94.07,doi.org/10.1093/nar/gky752,GSE111094,1.0
ytf__delyieP__2,yTF,ytf,delyieP,2,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655 del_yieP,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR6782006_1.fastq.gz,ytf/SRR6782006_2.fastq.gz,ytf__delyieP__2.bam,94.80,doi.org/10.1093/nar/gky752,GSE111094,1.0


,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
ytf__wt_glc__1,yTF,ytf,wt_glc,1,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR6781997_1.fastq.gz,ytf/SRR6781997_2.fastq.gz,ytf__wt_glc__1.bam,95.74,doi.org/10.1093/nar/gky752,GSE111094,1.0
ytf__wt_glc__2,yTF,ytf,wt_glc,2,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR6781998_1.fastq.gz,ytf/SRR6781998_2.fastq.gz,ytf__wt_glc__2.bam,97.01,doi.org/10.1093/nar/gky752,GSE111094,1.0


,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
ytf__wt_ph5__1,yTF,ytf,wt_ph5,1,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR7057985_1.fastq.gz,ytf/SRR7057985_2.fastq.gz,ytf__wt_ph5__1.bam,94.96,doi.org/10.1093/nar/gky752,GSE111094,1.0
ytf__wt_ph5__2,yTF,ytf,wt_ph5,2,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR7057986_1.fastq.gz,ytf/SRR7057986_2.fastq.gz,ytf__wt_ph5__2.bam,94.67,doi.org/10.1093/nar/gky752,GSE111094,1.0


,study,project_id,condition_id,rep_id,contact,library creator,run_date,Strain Description,Organism,Base Media,...,Sequencing Machine,Additional Details,Biological Replicates,R1,R2,BAM,alignment,DOI,GEO,Public
sample_id,,,,,,,,,,,,,,,,,,,,,
ytf__wt_ph8__1,yTF,ytf,wt_ph8,1,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR7057987_1.fastq.gz,ytf/SRR7057987_2.fastq.gz,ytf__wt_ph8__1.bam,94.96,doi.org/10.1093/nar/gky752,GSE111094,1.0
ytf__wt_ph8__2,yTF,ytf,wt_ph8,2,Ye Gao,Ye Gao,10/2/2017 0:00,Escherichia coli K-12 MG1655,MG1655,M9,...,HiSeq 4000,NaN,2,ytf/SRR7057988_1.fastq.gz,ytf/SRR7057988_2.fastq.gz,ytf__wt_ph8__2.bam,94.50,doi.org/10.1093/nar/gky752,GSE111094,1.0


In [7]:
# add a biological replicates count column to the metadata
# this is needed later on
for name, group in ica_data.metadata.groupby(['project_id', 'condition_id']):
    ica_data.metadata.loc[group.index, 'Biological Replicates'] = group.shape[0]

### Gene info (GENE_DIR+'gene_info.csv')

In [8]:
# ADDRESS THESE IF THEY FAIL
# has a gene name column
assert('gene_name' in ica_data.gene_info.columns)

# can match to the data
assert(ica_data.X.index.isin(ica_data.gene_info.index).all())
assert(ica_data.S.index.isin(ica_data.gene_info.index).all())

Check for gene names. We expect some genes to be named, but we do not require all of them to be. The gene_name column for an unnamed gene will be the named the same as its locus tag in the following code block.

In [9]:
# MAKE SURE THE OUTPUT IS EXPECTED FOR YOU
not_in = 0 # counts the missing gene loci
had_nan = 0 # counts the unassigned gene names
name = 0 # counts the genes with everything
for g in ica_data.S.index:
    if not(g in ica_data.num2name.keys()):
        ica_data.num2name[g] = g # assigns the locus tag as the name
        not_in += 1
    elif not(type(ica_data.num2name[g])==str):
        ica_data.num2name[g] = g # assigns the locus tag as the name
        had_nan += 1
    else:
        name += 1
no_name = not_in + had_nan
tot = no_name + name
print('%i (%.2f%%) genes had names.'%(name, name*100/tot))
print('%i (%.2f%%) genes were assigned their locus tag as a name.'%(not_in, not_in*100/tot))

3923 (100.00%) genes had names.
0 (0.00%) genes were assigned their locus tag as a name.


## Optional: Gene Links

This part is potentially the trickiest. We offer 'additional information' on our gene pages, which links each gene to a more informative database. For example:

- E. coli: [EcoCyc](https://ecocyc.org/)
- S. aureus: [AureoWiki](https://aureowiki.med.uni-greifswald.de/Main_Page)
- B. subtilis: [SubtiWiki](http://subtiwiki.uni-goettingen.de/)

If you would like to use this feature, you need to create a dictionary that takes gene loci as keys and returns links as items. There are two ways to do that:

1. Create a .csv file in which one column is the gene loci and the other is the corresponding links
2. Write a function that produces the links. The example from bacillus is shown.

If this organism is new to the website and it has a database, play around with the database and urls to see which option is best for you. If this organism is already online, use the same option as the other datasets for this organism.

### Gene links option 1: gene_links.csv

If you are choosing this option, name your file 'gene_links.csv' and place it in the organism annotation folder, but outside of the gene_files folder. This is because the gene_files folder is downloadable from the site, but we don't want people to download this links file.

In [10]:
# IF USING THIS, PLACE YOUR FILE AT THIS PATH
# YOU MAY NEED TO READ IT IN DIFFERENTLY DEPENDING ON FORMATTING
links = pd.read_csv('../annotation/gene_links.csv', index_col=0, header=None).to_dict()[1]
links

{'b0002': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG10998',
 'b0003': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG10999',
 'b0004': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG11000',
 'b0005': 'https://ecocyc.org/gene?orgid=ECOLI&id=G6081',
 'b0006': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG10011',
 'b0007': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG11555',
 'b0008': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG11556',
 'b0009': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG11511',
 'b0010': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG11512',
 'b0011': 'https://ecocyc.org/gene?orgid=ECOLI&id=G6082',
 'b0014': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG10241',
 'b0015': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG10240',
 'b0016': 'https://ecocyc.org/gene?orgid=ECOLI&id=G6083',
 'b0018': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG10373',
 'b0019': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG10652',
 'b0020': 'https://ecocyc.org/gene?orgid=ECOLI&id=EG11078',
 'b0021': 'https://ecocyc.org/gene?orgid=ECOLI

### Gene links option 2: get_links() function

In [ ]:
# IF USING THIS, MODIFY THIS FUNCTION
def get_links(locus):
    # by looking at the db, I noticed that I could construct my own 
    # search url for it
    prefix = 'http://subtiwiki.uni-goettingen.de/v3/gene/search/exact/'
    return prefix+locus

# use the function to fill in a links dictionary
links = {}
for g in ica_data.gene_info.index:
    links[g] = get_links(g)

### Gene links option 3: No links

In [ ]:
# IF USING THIS OPTION, UNCOMMENT AND RUN THE FOLLOWING
#links = {}

### Test your gene links

This may take some time (~25 min for E. coli), but we want to make sure the links work -- broken links are not cool on a website.

In [47]:
# IF USING GENE LINKS, MAKE SURE THIS OUTPUT MATCHES YOUR EXPECTATIONS
import requests
from tqdm import tqdm_notebook as tqdm
had_link = 0
no_link = 0
bad_links = pd.Series()
for g in tqdm(ica_data.S.index):
    if (g in links.keys()) and (type(links[g]) == str):
        request = requests.get(links[g])
        if request.status_code == 200: # good link
            had_link += 1
        else: # bad link
            bad_links.loc[g] = links[g] # store so you can fix
            links[g] = np.nan # remove the link from links dictionary
    else:
        no_link += 1
num_genes = ica_data.S.shape[0]
print('%i (%.2f%%) genes have working links.'%(had_link, had_link*100/num_genes))
print('%i (%.2f%%) genes don\'t have links.'%(no_link, no_link*100/num_genes))
print('%i (%.2f%%) genes had broken links!'%(bad_links.shape[0], bad_links.shape[0]*100/num_genes))


3865 (98.52%) genes have working links.
58 (1.48%) genes don't have links.
0 (0.00%) genes had broken links!


## TF links

When an iModulon is annotated (in curated_enrichments.csv) as having enriched transcription factors, we convert those TF strings into links. For all organisms except E. coli, we use the same gene database to assign those links. In E. coli, we prefer [RegulonDB](http://regulondb.ccg.unam.mx/) over EcoCyc, since the details there are more specific to regulatory activity. 

Note that the 'links' dictionary has LOCI has keys (ex: b0010), whereas tfs in the tf strings have names (ex: glpR). This will need to be reflected in the dictionaries.

In [11]:
# CHOOSE TF LINKS METHOD BELOW

# if using the same links 
# IF USING, DOUBLE CHECK BEHAVIOR
#tf_links = {tf:links[ica_data.name2num[tf]] for tf in ica_data.gene_info.gene_name}

# if reading in the links
tf_links = pd.read_csv('../annotation/tf_links.csv', index_col = 1, header=None).to_dict()[2]

# if using no tf_links
#tf_links = {}

The code block below will print the HTML string that we will use for each iModulon. It will show 'nan' if the iModulon has no assigned regulator, plain text if there are no links, and a string with urls in it if the string has links. Make sure that the regulators you expect to have links do have links in this output.

In [15]:
# TEST: MAKE SURE OUTPUT MAKES SENSE

for tf in enrich.TF:
    # if desired, find and modify the below function in general_functions.py
    print(tf_with_links(tf, tf_links))

nan
nan
nan
<a href="http://regulondb.ccg.unam.mx/regulon?term=ECK120015690&organism=ECK12&format=jsp&type=regulon" target="_blank">dhaR</a> or <a href="http://regulondb.ccg.unam.mx/regulon?term=ECK120011240&organism=ECK12&format=jsp&type=regulon" target="_blank">mlc</a>
histidine
Sigma28
<a href="http://regulondb.ccg.unam.mx/regulon?term=ECK120013750&organism=ECK12&format=jsp&type=regulon" target="_blank">cbl</a> and <a href="http://regulondb.ccg.unam.mx/regulon?term=ECK120011312&organism=ECK12&format=jsp&type=regulon" target="_blank">cysB</a>
nan
nan
nan
<a href="http://regulondb.ccg.unam.mx/regulon?term=ECK125145850&organism=ECK12&format=jsp&type=regulon" target="_blank">btsR</a> or <a href="http://regulondb.ccg.unam.mx/regulon?term=ECK125158264&organism=ECK12&format=jsp&type=regulon" target="_blank">ypdB</a> or <a href="http://regulondb.ccg.unam.mx/regulon?term=ECK120012246&organism=ECK12&format=jsp&type=regulon" target="_blank">pdhR</a>
nan
nan
nan
<a href="http://regulondb.ccg.un

## Loop through all iModulons & Generate Files

This is where errors are most likely to occur. This website is a work in progress - please email krychel@eng.ucsd.edu if you run into issues here.

In [23]:
def make_directory(ica_data, k, row, links, tf_links, base_conds):
    
    # generate the plot files
    gene_table = gene_table_df(ica_data, k, row, links = links)
    gene_hist = gene_hist_df(ica_data, k, row)
    gene_scatter = gene_scatter_df(ica_data, k, base_conds, links)
    act_bar = activity_bar_df(ica_data, k, ica_data.metadata.reset_index())

    reg_venn = regulon_venn_df(ica_data, k, row)
    reg_scatter = regulon_scatter_df(ica_data, k, row)

    # generate a basic data df
    res = pd.Series(index=['name', 'TF', 'Regulator',
                           'Function', 'Category', 
                           'has_venn', 'scatter', 'has_meme'])
    res.loc['name'] = row.loc['name']
    res.loc['TF'] = row.TF
    res.loc['Regulator'] = tf_with_links(row.TF, tf_links)
    res.loc['Function'] = row.Function
    res.loc['Category'] = row.Category
    res.loc['has_venn'] = not(reg_venn is None)
    if reg_scatter is None:
        res.loc['scatter'] = 0
    else:
        res.loc['scatter'] = reg_scatter.shape[1] - 1
    res.loc['has_meme'] = False #memes will be added to the website later.

    # save output
    folder = 'iModulon_files/'+str(k)+'/'
    if not(os.path.isdir('iModulon_files')):
        os.mkdir('iModulon_files')
    if not(os.path.isdir(folder)):
        os.mkdir(folder)
    res.to_csv(folder+str(k)+'_meta.csv')
    gene_table.to_csv(folder+str(k)+'_gene_table.csv')
    gene_hist.to_csv(folder+str(k)+'_gene_hist.csv')
    gene_scatter.to_csv(folder+str(k)+'_gene_scatter.csv')
    act_bar.to_csv(folder+str(k)+'_activity_bar.csv')
    if not(reg_venn is None):
        reg_venn.to_csv(folder+str(k)+'_reg_venn.csv')
    if not(reg_scatter is None):
        reg_scatter.to_csv(folder+str(k)+'_reg_scatter.csv')
    ica_data.S[k].to_csv(folder+str(k)+'_gene_weights.csv')
    ica_data.A.loc[k].to_csv(folder+str(k)+'_activity.csv')

In [33]:
# suppress some warnings
from scipy.optimize import OptimizeWarning
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore', category=OptimizeWarning, append=True)
pd.options.mode.chained_assignment = None

In [37]:
# generate all files
for k, row in enrich.iterrows():
    make_directory(ica_data, k, row, links, tf_links, base_conds)

MISSING TF: histidine
MISSING TF: histidine
MISSING TF: Sigma28
MISSING TF: Sigma28
MISSING TF: Sigma38
MISSING TF: Sigma38
MISSING TF: Sigma32
MISSING TF: Sigma32
MISSING TF: gutR
MISSING TF: gutR
MISSING TF: Sigma54
MISSING TF: Sigma54
MISSING TF: Sigma54
MISSING TF: Sigma54
MISSING TF: Sigma19
MISSING TF: Sigma19


The output above will indicate if any of the TFs did not have genes with expression profiles associated with them. You can ignore missing TF expression profiles, unless you think a TF listed should have a gene and expression profile. If so, add the tf to ica_data.name2num with its appropriate index in X.

## Loop through all genes & Generate Files

In [66]:
# make these two data structures

# generate df with all iM genes indicated T/F 
im_genes= ica_data.S.T.copy()
for k,row in ica_data.S.T.iterrows():
    im_genes.loc[k,:] = abs(row) > abs(ica_data.thresholds[k])
    
# iM_table: start with main component of iM page
im_table = enrich[['name', 'Regulator', 'Function', 'Category']]
im_table = im_table.rename(columns={'name':'Name'})
im_table.index.name = 'k'

In [71]:
# IF USING LINKS, PUT THE DATABASE NAME HERE
gene_db = 'EcoCyc'

def make_gene_files(ica_data, g, row, links, im_table, im_genes, gene_db):
    
    # generate the activity bar df
    act_bar = gene_activity_bar_df(ica_data, g, ica_data.metadata.reset_index())
    
    # generate the iModulon table
    perGene_table = im_table.copy()
    perGene_table.insert(column ='in_iM', value = im_genes.loc[:, g], loc = 1)
    perGene_table.insert(column ='gene_weight', value = ica_data.S.loc[g, :], loc = 2)
    perGene_table = (perGene_table.assign(A=perGene_table['gene_weight'].abs())
            .sort_values(['in_iM','A'],ascending=[False, False])
            .drop('A', 1))
    
    # generate metadata df
    res = pd.Series(index=['gene_id', 'name', 'operon', 'product',
                           'cog', 'regulator(s)', 'link'])
    res.loc['gene_id'] = g
    res.loc['name'] = row.gene_name
    res.loc['operon'] = row.operon
    res.loc['product'] = row['product']
    res.loc['cog'] = row.cog
    res.loc['regulator(s)'] = ", ".join(ica_data.trn[ica_data.trn.gene_id == g].TF.to_list())
    res.loc['link'] = '<a href="' + str(links[g]) + '">'+ gene_db + '</a>'
    res.fillna(value="<i> Not Available </i>", inplace = True) 
    
    # save output
    folder = 'gene_page_files/'+str(g)+'/'
    if not(os.path.isdir('gene_page_files')):
        os.mkdir('gene_page_files')
    if not(os.path.isdir(folder)):
        os.mkdir(folder)
    
    res.to_csv(folder+str(g)+'_meta.csv', header=True)
    act_bar.to_csv(folder+str(g)+'_activity_bar.csv')
    ica_data.X.loc[g].to_csv(folder+str(g)+'_expression.csv')
    perGene_table.to_csv(folder + str(g) + '_perGene_table.csv')

In [73]:
# Filter out genes with no ICA data
reduced_gene_info = ica_data.gene_info.copy()
reduced_gene_info = reduced_gene_info[reduced_gene_info.index.isin(ica_data.X.index)]

# generate files for each gene
for g, row in reduced_gene_info.iterrows():
    make_gene_files(ica_data, g, row, links, im_table, im_genes, gene_db)

In [47]:
# make json for gene search

# Get df in correct format
gene_df = reduced_gene_info.copy()
gene_df["gene_id"] = reduced_gene_info.index
gene_df = gene_df[['gene_name', "gene_id", "product"]]
gene_df = gene_df.sort_values(by="gene_name").fillna(value = "not available")

#create gene info json
gene_df.to_json('./gene_page_files/gene_list.json', orient="records")

## Dataset home: iModulon table

Now, we have generated all the detailed individual pages. Let's generate a table of iModulons from curated_enrichments.csv, which will be the main part of the dataset home page.

Default behavior is to sort the iModulons by category. You can use whatever category names you choose, but make sure the list in cat_order is exhaustive if you want to use this feature. You can also fill in the 'category_num' column with individual indices to put your iModulons in any specific order of your choice.

In [50]:
# iM_table: this is the main component of the dataset page

# CHOOSE THE ORDER YOU WOULD LIKE YOUR CATEGORIES IN
cat_order = ['Carbon Source Utilization', 
             'Amino Acid and Nucleotide Biosynthesis',
             'Energy Metabolism',
             'Metal Homeostasis',
             'Miscellaneous Metabolism',
             'Structural Components',
             'Stress Response',
             'Regulator Discovery',
             'Biological Enrichment',
             'Genomic Alterations',
             'Uncharacterized']
cat_dict = {cat_order[i]:i for i in range(len(cat_order))}

# make the table - note the required columns for enrich are here.
im_table = enrich[['name', 'Regulator', 'Function', 'Category', 'n_genes', 'precision', 'recall']]
im_table = im_table.rename(columns={'name':'Name'})
im_table.Category = im_table.Category.fillna('Uncharacterized')
im_table.index.name = 'k'
im_table['category_num'] = [cat_dict[im_table.Category[k]] for k in im_table.index]
im_table.to_csv('iM_table.csv')

## General Dataset Details

Almost done! Fill in these last bits of information for display on your dataset page.

Note that in HTML, text can be formatted [like this](https://www.w3schools.com/html/html_formatting.asp). It is good practice to italicize organism names and use the correct capitalization (<i>Genus species</i>).

The dataset_meta Series is read very simply in javascript - if you would like to add any additional notes in this space, feel free to append to the series (for example, you could include a link to another site that you may want to have here, or other project details).

In [51]:
# dataset page header

# FILL THESE DETAILS IN
title = '<i>E. coli</i> ROS TALE'
organism = '<i>Escherichia coli</i> K-12'
strain = 'MG1655 and BW25113'

# get these statistics
num_genes = ica_data.X.shape[0]
num_samps = ica_data.X.shape[1]
num_conds = 0;
for n, g in ica_data.metadata.groupby(['condition_id', 'project_id']):
    num_conds += 1
num_ims = ica_data.S.shape[1]

dataset_meta = pd.Series({'Title':title,
                          'Organism': organism,
                          'Strain': strain,
                          'Number of Samples': num_samps,
                          'Number of Unique Conditions': num_conds,
                          'Number of Genes':num_genes,
                          'Number of iModulons': num_ims})
dataset_meta.to_csv('dataset_meta.csv')

In [52]:
# num_ims - used so that the 'next iModulon' button doesn't overflow
file = open('num_ims.txt', 'w')
file.write(str(num_ims))
file.close()

## Setting up the splash page

The splash page, encoded by index.html, is the first thing we see when we go to the website. If you go to it now, you'll notice only the links to public datasets, even though we have now generated everything for these pages to exist. The following code will produce a bit of html that needs to be put in the right place in index.html.

In [55]:
# splash page
# FILL OUT THESE DETAILS
large_title = '<i>E. coli</i> ROS'
sub_title = 'Unpublished Reactive Oxygen Species Evolution Study'
author = 'Rychel, et al.'

cwd = os.getcwd().split('/')
organism_dir = cwd[-2]
dataset_dir = cwd[-1]

In [60]:
html = '<div class="col-md-4 col-lg-3">\n'
html += '\t<a role="button" class="btn btn-primary btn-outline-light btn-block d-flex flex-column private_set" href="dataset.html?organism='
html += organism_dir
html += '&dataset='
html += dataset_dir
html += '">\n\t\t<h2>'
html += large_title
html += '</h2>\n\t\t<p>'
html += sub_title
html += '</p>\n\t\t<p class="mt-auto">'
html += author
html += '</p>\n\t</a>\n</div>'

file = open('html_for_splash.html', 'w')
file.write(html)
file.close()

Now, open the file 'html_for_splash.html' and copy its contents. Find the index.html page (in the root directory of the site) and look for the comment around line 200 that says 'INSERT NEW DATASETS...'. Paste the html there.

### All Done!

Return to [the wiki](https://github.com/SBRG/modulytics/wiki/Adding-a-Private-Project) or try viewing the site.